# Deploy Model to AI Platform Predications

### Import Libraries and set up parameters

In [33]:
import os
import numpy as np
PROJECT = 'crazy-hippo-01' 
BUCKET = 'crazy-hippo-01' 
REGION = 'us-central1' 
MODEL_NAME = 'earnings'
MODEL_VERSION='v1'
MODEL_LOCATION='gs://crazy-hippo-01/ai-platform/ml_engine_training/earnings/export/savedmodel/'
INPUT_DATA_FILE='earnings_predict.json'

# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.1'
os.environ['MODEL_NAME'] = MODEL_NAME
os.environ['MODEL_VERSION'] = MODEL_VERSION
os.environ['MODEL_LOCATION'] = MODEL_LOCATION
os.environ['INPUT_DATA_FILE'] = INPUT_DATA_FILE

### List Model in Google Cloud Storage Bucket

In [42]:
%%bash
gsutil ls gs://crazy-hippo-01/ai-platform/ml_engine_training/earnings/export/savedmodel

gs://crazy-hippo-01/ai-platform/ml_engine_training/earnings/export/savedmodel/
gs://crazy-hippo-01/ai-platform/ml_engine_training/earnings/export/savedmodel/saved_model.pb
gs://crazy-hippo-01/ai-platform/ml_engine_training/earnings/export/savedmodel/assets/
gs://crazy-hippo-01/ai-platform/ml_engine_training/earnings/export/savedmodel/variables/


### Deploy Model to AI Plaform Predictions through the SDK

In [70]:
%%bash

#Check if Model already exists
if [[ $(gcloud ai-platform models list --region='global' --format='value(name)' | grep $MODEL_NAME) ]]; then
    echo "$MODEL_NAME already exists"
else
    # create model
    echo "Creating $MODEL_NAME"
    gcloud ai-platform models create $MODEL_NAME --regions=$REGION
fi

#Check if version already exists and remove in that case
if [[ $(gcloud ai-platform versions list --model $MODEL_NAME --region='global' --format='value(name)' | grep $MODEL_VERSION) ]]; then
    echo "Deleting already existing $MODEL_NAME:$MODEL_VERSION ... "
    gcloud ai-platform versions delete --model=$MODEL_NAME $MODEL_VERSION
    echo "Please run this cell again if you don't see a Creating message ... "
    sleep 10
fi

#Create New Version and/or Model and Deploy
echo "Creating $MODEL_NAME:$MODEL_VERSION"
gcloud ai-platform versions create $MODEL_VERSION --model=$MODEL_NAME --async \
       --region='global' --framework=tensorflow --python-version=3.7 --runtime-version=$TFVERSION \
       --origin=$MODEL_LOCATION --staging-bucket=gs://$BUCKET

earnings already exists
Creating earnings:v1


Using endpoint [https://ml.googleapis.com/]
Using endpoint [https://ml.googleapis.com/]
Using endpoint [https://ml.googleapis.com/]


### Display list of Versions for the model

In [3]:
%%bash
# create model
gcloud ai-platform versions list --region='global' --model='earnings'

NAME  DEPLOYMENT_URI                                                                  STATE
v1    gs://crazy-hippo-01/ai-platform/ml_engine_training/earnings/export/savedmodel/  READY


Using endpoint [https://ml.googleapis.com/]


### Make a sample prediction

In [53]:
%%bash
gcloud ai-platform predict --model $MODEL_NAME --version $MODEL_VERSION --json-request $INPUT_DATA_FILE --region='global'

OUTPUT
[0.45906123518943787]


Using endpoint [https://ml.googleapis.com/]


#### JSON file as example

In [52]:
%%writefile earnings_predict.json
{
  "instances": [
    {"age" : 0.27397260273972607,
    "education_num" : 0.5333333333333333,
    "capital_gain" : 0.0,
    "workclass_Federal-gov" : 0,
    "workclass_Local-gov" : 0,
    "workclass_Never-worked" : 0,
    "workclass_Private" : 1,
    "workclass_Self-emp-inc" : 0,
    "workclass_Self-emp-not-inc" : 0,
    "workclass_State-gov" : 0,
    "workclass_Without-pay" : 0,
    "gender_Female" : 0,
    "gender_Male" : 1,
    "occupation_Adm-clerical" : 0,
    "occupation_Armed-Forces" : 0 ,
    "occupation_Craft-repair" : 0,
    "occupation_Exec-managerial" : 1,
    "occupation_Farming-fishing" : 0,
    "occupation_Handlers-cleaners" : 0,
    "occupation_Machine-op-inspct" : 0,
    "occupation_Other-service" : 0,
    "occupation_Priv-house-serv" : 0,
    "occupation_Prof-specialty" : 0,
    "occupation_Protective-serv" : 0,
    "occupation_Sales" : 0,
    "occupation_Tech-support" : 0,
    "occupation_Transport-moving" : 0,
    "marital_status_Divorced" : 0,
    "marital_status_Married-AF-spouse" : 1,
    "marital_status_Married-civ-spouse" : 0,
    "marital_status_Married-spouse-absent" : 0,
    "marital_status_Never-married" : 0,
    "marital_status_Separated" : 0,
    "marital_status_Widowed" : 0,
    "relationship_Husband" : 0,
    "relationship_Not-in-family" : 0,
    "relationship_Other-relative" : 0,
    "relationship_Own-child" : 0,
    "relationship_Unmarried" : 1,
    "relationship_Wife" : 0}
  ]
}

Overwriting earnings_predict.json


### Example of Python code for Client Library

In [ ]:
import googleapiclient.discovery

def predict_json(project, model, instances, version=None):
    """Send json data to a deployed model for prediction.

    Args:
        project (str): project where the Cloud ML Engine Model is deployed.
        model (str): model name.
        instances ([Mapping[str: Any]]): Keys should be the names of Tensors
            your deployed model expects as inputs. Values should be datatypes
            convertible to Tensors, or (potentially nested) lists of datatypes
            convertible to tensors.
        version: str, version of the model to target.
    Returns:
        Mapping[str: any]: dictionary of prediction results defined by the
            model.
    """
    # Create the ML Engine service object.
    # To authenticate set the environment variable
    # GOOGLE_APPLICATION_CREDENTIALS=<path_to_service_account_file>
    service = googleapiclient.discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']

In [ ]:
predict_json(PROJECT, MODEL_NAME, INPUT_DATA_FILE, MODEL_VERSION)